In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder , StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv('../input/loan-prediction-based-on-customer-behavior/Training Data.csv')
test = pd.read_csv('../input/loan-prediction-based-on-customer-behavior/Test Data.csv')
submission = pd.read_csv('../input/loan-prediction-based-on-customer-behavior/Sample Prediction Dataset.csv')

In [ ]:
train.head()

## Division of features into categorical and numeric

In [ ]:
cat_cols = [
            'Married/Single',
            'House_Ownership',
            'Car_Ownership',
            'STATE',
            'CITY',
            'Profession'
    ]
num_cols = [
            'Income',
            'Age',
            'Experience',
            'CURRENT_JOB_YRS',
            'CURRENT_HOUSE_YRS'
]

## deleting a client with missing values

In [ ]:
train = train.drop(index = [184204]);

## Let's start with visualization

Let's look at how people pay off a loan in a married (not married)

In [ ]:
sns.countplot(x = train['Married/Single'] , hue = train['Risk_Flag']);

Let's look at the distribution of the age of the bank's customers

In [ ]:
sns.histplot(train['Age']);

the main values of the age of the bank's customers

In [ ]:
print('main values of the age : ' , train['Age'].describe())

Let's look at various combinations of features with the target variable

In [ ]:
sns.countplot(train['Car_Ownership'] , hue = train['Risk_Flag']);

In [ ]:
sns.countplot(train['House_Ownership'] , hue = train['Risk_Flag']);

In [ ]:
sns.countplot(train['Experience'] , hue = train['Risk_Flag']);

Let's look at the distribution of the target variable

In [ ]:
sns.countplot(train['Risk_Flag'])

Classes are not balanced

In [ ]:
value_counts = train['Risk_Flag'].value_counts()
value_counts[0] / value_counts[1]

## Preprocessing categorial features

In [ ]:
le = LabelEncoder()
cat_features = [
                'Married/Single',
                'House_Ownership',
                'Car_Ownership',
]
for col in cat_features:
  train[col] = le.fit_transform(train[col])
  train[col] = train[col].astype(int)
  test[col] = le.fit_transform(test[col])
  test[col] = test[col].astype(int)

In [ ]:
train

In [ ]:
X=train[['Income', 'Age', 'Experience', 'Married/Single',
       'House_Ownership', 'Car_Ownership',
       'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS']]
y=train['Risk_Flag']

using smote, we generate synthetic data for balancing classes

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , train_size = 0.8)
smote = SMOTE(sampling_strategy = 0.85)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
X_train.shape , y_train.shape

## Modeling 

In [ ]:
model = RandomForestClassifier(n_estimators= 650 , n_jobs= -1)
model.fit(X_train ,y_train)
y_pred = model.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
output = pd.DataFrame({'id': X_test.index,
                       'risk_flag': y_pred})
output.to_csv('submission.csv', index=False)